# Sprint25課題 Seq2Seq

## 1.この課題の目的

- RNNの活用例を知る

## 2.機械翻訳

RNNの最も基本的な活用例としては機械翻訳がある。

これは時系列データを入力し、時系列を出力するSequence to Sequenceの手法によって実現できる。

### 【問題1】機械翻訳の実行とコードリーディング
Keras公式のサンプルコードを実行し、短い英語からフランス語への変換を行う。

[keras/lstm_seq2seq.py at master · keras-team/keras](https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py)

その上でこのサンプルコードの各部分がどういった役割かを読み取り、まとめること。

以下のように、どこからどこの行が何をしているかを記述する。

（例）

- 51から55行目 : ライブラリのimport
- 57から62行目 : ハイパーパラメータの設定

In [0]:
from google.colab  import drive
drive.mount("gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at gdrive


In [0]:
import os, sys

import numpy as np

from keras.models import Model
from keras.layers import Input, LSTM, Dense, CuDNNLSTM

from PIL import Image

In [0]:
folder_path =  '/content/gdrive/My Drive/My Documents/Study/Programming/DIVE INTO CODE/Class/sprint25'
im2seq_path = '/content/gdrive/My Drive/My Documents/Study/Programming/DIVE INTO CODE/Class/sprint25/image_captioning'

In [0]:
os.chdir(folder_path)

#### Data Source
- [Tab-delimited Bilingual Sentence Pairs ](http://www.manythings.org/anki/)
- コードの役割は、コード中にコメントとして記載した。

In [0]:
batch_size = 64
epochs = 10
latent_dim = 256 # 符号化空間の潜在次元
num_samples = 10000 # 使用するテキスト数の上限
data_path = os.path.join(folder_path, 'fra-eng/fra.txt') # データ格納先

# Vectorize the data.
input_texts = [] # 英語
target_texts = [] # 仏語
input_characters = set()
target_characters = set()


with open(data_path, 'r', encoding='utf-8') as f:
    # 英仏テキストの組み合わせを改行文字（"\n"）で分割
    lines = f.read().split('\n')
# 最大num_samples個のテキストの組み合わせを、入力値とターゲットに分割する
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # 水平タブ（"\t"）をスタート、改行（"\n"）をターゲット文章の終わりとする
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    # 入力、およびターゲットの単語リストを作成
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

# 生成した単語リストをアルファベット順にソートする
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

# 語彙数
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

# 最も長いテキストの単語数をカウントする
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

# 単語をIDに変換する辞書
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# Enncoderのデータ（入力値の単語ベクトル）を保存する配列 shape(samples, sequence, vocab)を初期化
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')

# Decoderのデータ（ターゲットの単語ベクトル）を保存する配列 shape(samples, sequence, vocab)を初期化
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

# Decoderのデータ（ターゲットの単語ベクトル）を1シーケンスずらして保存する配列 shape(samples, sequence, vocab) => 推論に用いる
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

# 単語ベクトルの生成
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    # 入力値の単語ベクトルを生成
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # ターゲットの単語ベクトルを生成
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # ターゲットの単語ベクトルを1シーケンスずらして保存 => 推論に用いる
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

# モデルの構築（Encoder）
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = CuDNNLSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

# モデルの構築（Decoder）
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = CuDNNLSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# モデルの定義
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# コンパイル、学習
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

# モデルを保存
model.save('s2s.h5')


# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states) # encoder_inputs = Input(shape=(None, num_encoder_tokens)) & encoder_states = [state_h, state_c]

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# IDを単語に変換する辞書
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # 入力値をEncodingし、隠れ状態を予測する
    states_value = encoder_model.predict(input_seq)

    # Decodingしたテキストを格納する配列
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # 出力値の最初の文字は"水平タブ"とする
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

# 入力したテキスト（英語）をEncode/Decodeした後のテキスト（仏語）を出力する
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


W0628 06:27:18.475404 140470350116736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0628 06:27:18.507002 140470350116736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0628 06:27:21.029186 140470350116736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0628 06:27:21.510671 140470350116736 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0628 06:27:21.536674 140470350116736 deprecation_wrappe

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 8s 1ms/step - loss: 0.9140 - val_loss: 0.9270
Epoch 2/100
8000/8000 [==============================] - 2s 220us/step - loss: 0.7269 - val_loss: 0.7571
Epoch 3/100
8000/8000 [==============================] - 2s 221us/step - loss: 0.6180 - val_loss: 0.6755
Epoch 4/100
8000/8000 [==============================] - 2s 219us/step - loss: 0.5621 - val_loss: 0.6249
Epoch 5/100
8000/8000 [==============================] - 2s 223us/step - loss: 0.5232 - val_loss: 0.5930
Epoch 6/100
8000/8000 [==============================] - 2s 225us/step - loss: 0.4922 - val_loss: 0.5702
Epoch 7/100
8000/8000 [==============================] - 2s 223us/step - loss: 0.4674 - val_loss: 0.5400
Epoch 8/100
8000/8000 [==============================] - 2s 224us/step - loss: 0.4457 - val_loss: 0.5224
Epoch 9/100
8000/8000 [==============================] - 2s 225us/step - loss: 0.4264 - val_loss: 0.5120
Epoch 10/

/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer cu_dnnlstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'cu_dnnlstm_1/strided_slice_16:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'cu_dnnlstm_1/strided_slice_17:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Run!
Decoded sentence: Cours !

-
Input sentence: Who?
Decoded sentence: Qui ?

-
Input sentence: Wow!
Decoded sentence: Ça alors !

-
Input sentence: Fire!
Decoded sentence: Au feu !

-
Input sentence: Help!
Decoded sentence: À l'aide !

-
Input sentence: Jump.
Decoded sentence: Saute.

-
Input sentence: Stop!
Decoded sentence: Stop !

-
Input sentence: Stop!
Decoded sentence: Stop !

-
Input sentence: Stop!
Decoded sentence: Stop !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Hello!
Decoded sentence: Bavou !

-
Input sentence: Hello!
Decoded sentence: Bavou !

-
Input sentence: 

- 英仏翻訳が機能していることを確認した。

## 3.イメージキャプショニング

- 他の活用例としてイメージキャプショニングがある。（画像に対する説明の文章を推定するタスク）

- これは画像を入力し、時系列を出力するImage to Sequenceの手法によって行われる。

 [pytorch-tutorial/tutorials/03-advanced/image_captioning at master · yunjey/pytorch-tutorial](https://github.com/yunjey/pytorch-tutorial/tree/master/tutorials/03-advanced/image_captioning)

- イメージキャプショニングは学習に多くの時間がかかるため、ここでは学習済みの重みが公開されている実装を動かす。

- Kerasには平易に扱える実装が公開されていないため、今回はPyTorchによる実装を扱う。



### 【問題2】イメージキャプショニングの学習済みモデルの実行

- 上記実装において、**5. Test the model**の項目を実行し、また、自身で用意した画像に対しても文章を生成する。

- これらに対してどういった文章が出力されたかを記録して提出すること。

- データセットからの学習は行わず、学習済みの重みをダウンロードして利用する。

- 注意点：デフォルトで設定されている重みのファイル名とダウンロードできる重みのファイル名は異なっているため、書き換える必要がある。

### Image Captioning

- Image Captioningの目的：与えられた入力画像を自然言語の記述に変換する
- Image Encoder：畳み込みニューラルネットワーク（Convolutional Neural Network）
- Model：ILSVRC-2012-CLS画像分類データセットで事前学習されたresnet-152モデルを使用 
- Decoder：Long short term memory Network

 ![Model Captioning](https://github.com/yunjey/pytorch-tutorial/raw/master/tutorials/03-advanced/image_captioning/png/model.png)

In [0]:
os.chdir(im2seq_path)
os.getcwd()

'/content/gdrive/My Drive/My Documents/Study/Programming/DIVE INTO CODE/Class/sprint25/image_captioning'

### サンプルコードを動かす
- [画像元] (https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/03-advanced/image_captioning/png/example.png)
- ![キリン](https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/03-advanced/image_captioning/png/example.png?raw=true) 

In [0]:
!python sample.py --image="./image/example.png"

<start> a group of giraffes standing next to each other . <end>


### 用意した画像を自然言語に変換する

- [参照] [PAKUTASO（フリー素材）](https://www.pakutaso.com/20160230047post-6972.html)

 <画像１：ウユニ塩湖>
 ![代替テキスト](https://www.pakutaso.com/shared/img/thumb/MIYA160131480493_TP_V.jpg)

 <画像２：水鏡に映る中尊寺弁財天堂>
 ![代替テキスト](https://www.pakutaso.com/shared/img/thumb/AME20181113A004_TP_V.jpg)

#### 生成した文章は以下のとおり。

In [0]:
print("Uyuni")
!python sample.py --image="./image/uyuni1.jpg"

print("\nTemple")
!python sample.py --image="./image/temple1.jpg"

Uyuni
<start> a view of a lake with a boat in the background . <end>

Temple
<start> a close up of a bench in the woods <end>


### 【問題3】Kerasで動かしたい場合はどうするかを調査

PyTorchによる実装を動かしたが、何らかの理由からKerasで動かしたい状況が考えられる。

- どういった手順を踏むことになるか調査し、できるだけ詳しく説明すること。

- 特に、今回はPyTorchのための学習済みの重みをKerasで使えるようにしたいので、その点については必ず触れること。

#### PytorchによるモデルをKerasで動かす方法

1. PytorchによるモデルのアーキテクチャをKerasで再構築する。（レイヤーは１対１対応する）
  - 入力、出力の形状を揃える
  - カーネルサイズを揃える
  - ストライド、パディングの方法を揃える


2. Kerasで再構築したモデルの各レイヤーに対して、Pytorchのレイヤーの重みを共有する。
  - Pytorchのモデルに対して**.state_dict()メソッド**を適用し、各レイヤーの重みを抽出する。
      - アウトプットのデータ型は"pythonの辞書型"であり、keyはレイヤーの名称、valueは重みである。
  
  - Kerasのlayer.set_weights(weights)メソッドを用いて、Pytorchのモデルから抽出した重みを代入していく。注意点は以下の通り。
    - 重みを格納した辞書のkeyであるレイヤーの名称を、KerasとPytorchで揃えること。
    - Pytorchモデルで取得した重みのデータ型を、Pytorchの.numpy()メソッドでNumpy配列に変換すること。 （Pytorchから取得した重みの型はTensorに対して、set_weights()メソッドでkerasモデルに代入する重みの型はNumpyである必要がある）

### 参考にした情報

- [Keras official document](https://keras.io/ja/layers/about-keras-layers/)
- [Pytorch official document](https://pytorch.org/tutorials/beginner/saving_loading_models.html#what-is-a-state-dict)

#### Take away

1. 機械翻訳やImage Captioningの事例を通じて、自然言語処理におけるrnnの有用性（文脈理解）を理解した。


2. Image Captioningのように、複数領域（例えば、画像処理と自然言語処理を組み合わせる）にまたがるモデルを構築する際には、膨大なデータセット、学習時間を要するため、基本的には事前学習した重みを用いることが肝要（転移学習）。必要に応じてファインチューニングすると良い。


3.  異なるフレームワーク間でモデルの重みを共有する方法に関しても理解が深まった。論文実装で公開されているコードは、Kerasでしか構築されていない、あるいはPytorchでしか構築されていないこともある。実際の課題や企業の課題に応じて利用するフレームワークに制限がある場合も考えられ、その意味でも異なるフレームワークに書き換える経験を積み重ねたい。